In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

In [ ]:
trainDF = pd.read_csv("/kaggle/input/playground-series-s3e22/train.csv", header=0)
trainDF.head()

In [ ]:
trainDF.columns

In [ ]:
testDF = pd.read_csv("/kaggle/input/playground-series-s3e22/test.csv", header=0)
testDF.head()

In [ ]:
testDF.columns

In [ ]:
trainX = trainDF.drop('outcome', axis=1)
trainY = trainDF['outcome']

testX = testDF.copy()

In [ ]:
categoricalColumns = trainX.select_dtypes(include=['object']).columns

encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')

trainX_encoded = pd.DataFrame(encoder.fit_transform(trainX[categoricalColumns]), columns=encoder.get_feature_names_out())
testX_encoded = pd.DataFrame(encoder.transform(testX[categoricalColumns]), columns=encoder.get_feature_names_out())

In [ ]:
imputer = SimpleImputer(strategy='mean')

trainX_imputed = pd.DataFrame(imputer.fit_transform(trainX.select_dtypes(exclude=['object'])), columns=imputer.get_feature_names_out())
testX_imputed = pd.DataFrame(imputer.transform(testX.select_dtypes(exclude=['object'])), columns=imputer.get_feature_names_out())

In [ ]:
trainX_processed = pd.concat([trainX_encoded, trainX_imputed], axis=1)
testX_processed = pd.concat([testX_encoded, testX_imputed], axis=1)

In [ ]:
trainX_processed.head()

In [ ]:
classifier = RandomForestClassifier(random_state=42)

In [ ]:
classifier.fit(trainX_processed, trainY)

In [ ]:
yPred = classifier.predict(testX_processed)

In [ ]:
submission1 = pd.DataFrame({'id':testDF['id'], 'outcome':yPred})
submission1.head()

In [ ]:
submission1.to_csv('submission1.csv', index=False)

In [ ]:
import lightgbm as lgb

In [ ]:

classifier = lgb.LGBMClassifier(objective = 'multiclass',
    n_estimators=1000,
                     max_depth=10,
                     random_state=42)

In [ ]:
trainY = trainY.map({'died': 0, 'euthanized': 1, 'lived': 2})
trainY.head()

In [ ]:
classifier.fit(trainX_processed, trainY)

In [ ]:
yPred = classifier.predict(testX_processed)

In [ ]:
submission1 = pd.DataFrame({'id':testDF['id'], 'outcome':yPred})
submission1.head()

In [ ]:
submission1.to_csv('submission_baseline_lgbm.csv', index=False)